In [1]:
import numpy as np

In [2]:
def pretty_print(ideal):
    for eq in ideal.gens():
        print(eq)

In [3]:
class VectorOfMatrices:
    def __init__(self, matrices):
        self.matrices = matrices
        self.len = len(matrices)

    def get_len(self):
        return self.len
     
    def __mul__(self, other):
        # https://doc.sagemath.org/html/en/reference/matrices/sage/matrix/matrix_dense.html
        if not isinstance(other, sage.matrix.matrix_dense.Matrix_dense):
            raise TypeError("We only multiply by a matrix my dear")

        if other.ncols() != self.len:
            raise ValueError("Matrix dimensions do not match")

        # multiply them by hand
        result = [sum(other[i, j] * self.matrices[j] for j in range(self.len)) for i in range(other.nrows())]
        # gives back a list
        return result

    def __sub__(self, other):
        # https://doc.sagemath.org/html/en/reference/modules/sage/modules/vector_modn_dense.html#sage.modules.vector_modn_dense.Vector_modn_dense
        # if not isinstance(other, sage.modules.vector_modn_dense):
            # raise TypeError("We only subtract other VectorOfMatrices my dear")

        if other.len != self.len:
            raise ValueError("Vector dimensions do not match")

        result = [self.matrices[i] - other.matrices[i] for i in range(self.len)]
        return result
        
    def __repr__(self):
        return f"VectorOfMatrices(\n{self.matrices})"
    
    __rmul__ = __mul__
    __rsub__ = __sub__

In [32]:
class Instance:
    def __init__(self, n, m, p, S=None, T=None):
        self.n = n
        self.m = m
        
        if p % 2 == 0:
            print("We don't really like the char 2 because things cancel out i think")
        self.size = p
        self.field = GF(p)

        self.R = PolynomialRing(self.field, n^2 + m^2, 'x')
        # self.R.inject_variables(verbose=False)
        
        self.F = None
        self.S = S
        self.T = T
        
        self.P = None

        # print("--------------------------------------------------------------------------------------------------")
        # print(f"| An instance of inhQMLE over Finite Field with {self.size} elements, n = {self.n} variables and m = {self.m} equations. |")
        # print("--------------------------------------------------------------------------------------------------")

    def populate(self):
        self.generate_private()
        self.generate_public()
        
    def _random_symmetric_matrix(self):
        # get a random matrix
        A = random_matrix(self.field, self.n, self.n)
        # and make it symmetric
        return A + A.transpose()

    def _random_polynomial(self):
        return self._random_symmetric_matrix(), random_vector(self.field, self.n), self.field.random_element()

    def generate_private(self):
        if self.T == None:
            self.T = matrix(GL(self.m, self.field).random_element())
        if self.S == None:
            self.S = matrix(GL(self.n, self.field).random_element())
        
        if self.F == None:
            # print(f"Generating random polynomials over a finite field with {self.size} elements")
            self.F = [[],[],[]]
            for i in range(m):
                A, B, C = self._random_polynomial()
                self.F[0].append(A)
                self.F[1].append(B)
                self.F[2].append(C)
    
    def show_private(self):
        print(f"The input transformation S\n{self.S}")
        print(f"The output transformation T\n{self.T}")
        print("The central map F[i](x) = x^T A[i]x + b[i]x + c[i]")
        for i in range(m):
            print(f"F[{i}]:Matrix\n{self.F[0][i]}, Vector {self.F[1][i]}, Scalar {self.F[2][i]}\n---------------------------------------------")

    def generate_public(self):
        if self.P == None:
            self.P = [[],[],[]]
            v = VectorOfMatrices([self.S.transpose()*A*self.S for A in self.F[0]])
            self.P = [self.T*v, self.T*matrix([B*self.S for B in self.F[1]]), self.T*vector(self.F[2])]

    def show_public(self):
        print("The public set of polynomials P(x) = T F(Sx)")
        for i in range(m):
            print(f"P[{i}]: Matrix\n{self.P[0][i]}, Vector {self.P[1][i]}, Scalar {self.P[2][i]}\n---------------------------------------------")

    def model(self):
        gens = self.R.gens()

        # first n^2 elements of R2
        S_ = matrix(self.R, self.n, self.n, gens[:self.n^2])

        # we get the last m^2 variables of R2
        T_ = matrix(self.R, self.m, self.m, gens[-self.m^2:])

        auxiliary = [[],[],[]]
        
        auxiliary0 = T_*VectorOfMatrices(self.P[0])
        Q0 = VectorOfMatrices([S_.transpose()*A*S_ for A in self.F[0]])
        
        auxiliary1 = T_*self.P[1]
        Q1 = matrix([B*S_ for B in self.F[1]])
        
        auxiliary2 = T_*self.P[2]
        Q2 = vector(self.F[2])
        
        # add the field equations by hand
        system = [x^self.size - x for x in gens]
        M = VectorOfMatrices(auxiliary0) - Q0
        for i in range(m):
            system += M[i].list()
            system += (auxiliary1[i] - Q1[i]).list()
            system += [auxiliary2[i] - Q2[i]]
        
        # the list(set(.)) removes the duplicates
        return self.R.ideal(list(set(system)))

    # given a system of polynomials, check if it correctly corresponds
    # to the actual matrices it was created from
    #
    # ideal is an ideal of multivariate polynomial ring defined above
    def is_ideal_correct(self, ideal):

        # point = self.S.list() + matrix(GL(self.m, self.field).random_element()).list()
        point = self.S.list() + self.T.inverse().list()
        # point = self.S.list()+self.T.list()
    
        gens = self.R.gens()
        # make an appropriate dict
        substitution_dict = {gens[i]: point[i] for i in range(len(point))}
        
        return all(f.subs(substitution_dict) == 0 for f in ideal.gens())


    # i think this is wrong as the variety might have many points but 
    # we are only interested in the point corresponding to S and T
    def is_variety_correct(self, variety):
        point = self.S.list() + self.T.inverse().list()
        gens = self.R.gens()

        return any(variety[j][var] == point[i] for i, var in enumerate(gens) for j in range(len(variety)))
        # return all(f.subs(substitution_dict) == 0 for f in ideal.gens())

In [70]:
# we dont like p = 2 coz things cancel out
p = 3
n = 2
m = 2

T = identity_matrix(GF(p), m)
S = identity_matrix(GF(p), n)
inst = Instance(n, m, p, S, T)

inst.generate_private()
# inst.show_private()

inst.generate_public()
# inst.show_public()

I = inst.model()
# pretty_print(I)
V = sorted(I.variety(), key=str)
var = V[0]
gens = inst.R.gens()
point = S.list() + T.list()

# print(point)
substitution_dict = {gens[i]: point[i] for i in range(len(point))}
print(substitution_dict)
print(substitution_dict == var)
print(var)
# print(inst.is_ideal_correct(I))
# print(I.groebner_basis())

# %timeit I.groebner_basis()
# %timeit I.variety()

# print(inst.is_variety_correct(V))

{x0: 1, x1: 0, x2: 0, x3: 1, x4: 1, x5: 0, x6: 0, x7: 1}
False
{x7: 0, x6: 0, x5: 2, x4: 1, x3: 1, x2: 0, x1: 0, x0: 0}


In [212]:
primes = Primes()
p = primes.next(2)

i = 0
n = 5
m = 4

while (i < 5):
    i+=1
    inst = Instance(n, m, p)
    inst.populate()
    I = inst.model()
    if not inst.is_ideal_correct(I):
        print("oh nooooooo")

    print(timeit.eval("I.groebner_basis()"))
    inst = Instance(n, m, p^2)
    inst.populate()
    I = inst.model()
    if not inst.is_ideal_correct(I):
        print("oh nooooooo")

    print(timeit.eval("I.groebner_basis()"))
    
    inst = Instance(n, m, p^3)
    inst.populate()
    I = inst.model()
    if not inst.is_ideal_correct(I):
        print("oh nooooooo")

    print(timeit.eval("I.groebner_basis()"))

625 loops, best of 3: 293 ns per loop


KeyboardInterrupt: 